In [1]:
!pip install googletrans==3.1.0a0

  Using cached googletrans-3.1.0a0.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
     ---------------------------------------- 1.5/1.5 MB 10.9 MB/s eta 0:00:00
  Using cached h2-3.2.0-py2.py3-none-any.whl (65 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl (12 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl (38 kB)
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=99261524c893b2e7248ee61c39a35a590040d461aadf7b1363f18d2bba129875
  Stored in directory: c:\users\dimas\appdata\local\pip\cache\wheels\ae\e1\6c\5137bc3f35aa130deea71575e165cc4f4f0680a88f3d90a636
Successfully built googletrans
  A

In [4]:
import os
import json
import re
from tqdm import tqdm

# import fitz

import googletrans
import pandas as pd

# Читаем данные Contract Understanding Atticus Dataset (CUAD)

Оригинал доступен по [ссылке](https://zenodo.org/record/4595826).

In [5]:
with open('../data/CUAD_data/CUADv1.json') as f:
    data = json.load(f)

In [6]:
texts = [example['paragraphs'][0]['context'] for example in data['data']]

Разбиваем тексты на абзацы, так как переводчик принимает только небольшие порции текста.

In [7]:
texts_chunks = []
for text in texts:
    texts_chunks.append([chunk for chunk in text.split('\n\n') if chunk])

In [8]:
translator = googletrans.Translator()

Переводим тексты по абзацам и добавляем пустые строки туда, откуда они были удалены

In [9]:
results = []
for i, text_chunks in tqdm(enumerate(texts_chunks)):
    result = translator.translate(text_chunks, dest='ru', src='en')
    translated_chunks = [chunk_result.text for chunk_result in result]
    all_chunks = []
    j = 0
    for raw_chunk in texts[i].split('\n\n'):
        if raw_chunk:
            all_chunks.append(translated_chunks[j])
            j+=1
        else:
            all_chunks.append('')
    results.append('\n\n'.join(all_chunks))

109it [48:37, 26.77s/it]


KeyboardInterrupt: 

Добавляем данные по типу договоров

In [ ]:
contract_types_ru = []
contract_types_en = []
for i, example in tqdm(enumerate(data['data'])):
    contract_type = example['paragraphs'][0]['qas'][0]['answers'][0]['text']
    contract_type_ru = translator.translate(contract_type, dest='ru', src='en').text
    contract_types_ru.append(contract_type_ru)
    contract_types_en.append(contract_type)

Сохраняем данные в табличном формате

In [ ]:
pd_data = pd.DataFrame({'text_en': texts,
                        'text_ru': results,
                        'contract_types_en': contract_types_en,
                        'contract_types_ru': contract_types_ru})

# Проставляем метки

In [ ]:
filename2class = {}
for part_num in range(1, 4):
    for classname in os.listdir(f'../data/CUAD_data/full_contract_pdf/Part_{"I"*part_num}'):
        for filename in os.listdir(f'../data/CUAD_data/full_contract_pdf/Part_{"I"*part_num}/{classname}'):
            clear_filename = filename.lower()\
                                     .split('.pdf')[0]\
                                     .strip()\
                                     .strip('-')\
                                     .replace('agreement_option ', '')\
                                     .replace('é', 'e')
            clear_classname  = classname.split(' Agreement')[0]\
                                        .replace('Joint Venture _ Filing', 'Joint Venture')\
                                        .replace('Affiliate_Agreements', 'Affiliate')\
                                        .replace('License_Agreements', 'License')\
                                        .replace('Non_Compete_Non_Solicit', 'Non-Compete')
            filename2class[clear_filename] = clear_classname

In [ ]:
target = []

for ex in data['data']:
    clear_filename = ex['title'].lower()\
                                .strip()\
                                .strip('-')\
                                .replace('é', 'e')
    target.append(filename2class[clear_filename])
    
pd_data['target'] = target

In [ ]:
pd_data.head()

In [ ]:
pd_data.to_csv('../data/CUAD_data/translated_data.csv')